In [19]:
%pylab inline
import pg_toolkit as pgt
pgt.toolkit_config.set_pg_conn_string("dbname='decl'")
import hashlib
import glob
import os
from IPython.lib import backgroundjobs as bg
jobs = bg.BackgroundJobManager()
from IPython.display import Image, HTML, display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['record']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [20]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [21]:
# import json
# with open("feed.json", "r") as fp:
#     data = json.load(fp)
#     for record in data:
#         record['path'] = '/Users/tilarids/dev/decl/data/' + hashlib.sha224(record['declaration'].get('url','').encode('utf-8')).hexdigest() + '.pdf'

In [22]:
# pgt.import_json_into_pg("decls", data, lambda record: record['id'], create_table=True, skip_duplicates=True)

In [23]:
pgt.import_glob_list_into_pg('decls_pdfs', '/Users/tilarids/dev/decl/data/*.pdf')
data = pgt.pg_query("""
    SELECT
        t.id as id,
        t.data->'declaration'->'url' as url,
        t.data->'path' as path
    FROM decls t
    INNER JOIN decls_pdfs dp ON (t.data->>'path'=dp.path)
    WHERE t.data->'declaration'->>'url' != ''
""")

In [24]:
from extract_util import extract_borders

In [25]:
import cv2
def extract_fields():
    for path in log_progress(data['path']):
#         print "Processing path:", path,
#         import pdb; pdb.set_trace()
        for img_path in glob.glob('/Users/tilarids/dev/decl/img_data/' + os.path.basename(path) + ".*.png")[:2]:
            try:
                z = None
                z = pgt.pg_query_by_id('detected_borders', img_path)
            except:
                pass
            if z:
#                 print "skip",
                continue
            
            img = cv2.imread(img_path)
            height, width = img.shape[0], img.shape[1]
#             import pdb; pdb.set_trace()
            left_border, top_border, right_border, bottom_border = extract_borders(img)
            if left_border is None or right_border is None or top_border is None or bottom_border is None:
                pgt.import_json_into_pg('detected_borders', 
                                        [{'path':path, 'img_path':img_path}], 
                                        lambda record: record['img_path'], 
                                        create_table=False, 
                                        skip_duplicates=True)
                continue
            detected_width = right_border[1] - left_border[1]
            detected_height = bottom_border[2] - top_border[2]
            out = {'left_border':map(float,left_border),
                     'top_border':map(float,top_border),
                     'right_border':map(float,right_border),
                     'bottom_border':map(float,bottom_border),
                     'detected_width':detected_width,
                     'detected_height':detected_height,
                     'height': height,
                     'width': width,
                     'img_path':img_path,
                     'path':path}
#             import pdb; pdb.set_trace()
            pgt.import_json_into_pg("detected_borders", [out], lambda record: record['img_path'], create_table=False, skip_duplicates=True)
#         print ""

In [26]:
import subprocess
def apply_tesseract():
    for path in log_progress(data['path']):
#         print "Processing path with tesseract:", path,
        for img_path in glob.glob('/Users/tilarids/dev/decl/img_data/' + os.path.basename(path) + ".*.png")[:2]:
            try:
                z = None
                z = pgt.pg_query_by_id('tesseract_ocr', img_path)
            except:
                pass
            if z:
#                 print "skip",
                continue
            try:
                out = subprocess.check_output(["tesseract", img_path, "stdout", "-l", "ukr"])
            except:
                continue
            record = {'ocr': out,
                     'img_path':img_path,
                     'path':path}
#             import pdb; pdb.set_trace()
            pgt.import_json_into_pg("tesseract_ocr", [record], lambda record: record['img_path'], create_table=False, skip_duplicates=True)
#         print ""

In [27]:
jobs.new(extract_fields)
jobs.new(apply_tesseract)

Starting job # 0 in a separate thread.
Starting job # 2 in a separate thread.


<BackgroundJob #2: <function apply_tesseract at 0x106930c08>>

In [15]:
z = pgt.pg_query("""
    SELECT
        t.data->'ocr' as ocr,
        t.data->'img_path' as img_path
    FROM tesseract_ocr t
    WHERE t.data->>'img_path' LIKE '%.0.png' OR t.data->>'img_path' LIKE '%.1.png'
""")
z['ocrl'] = z['ocr'].map(lambda x: x.lower())
imgs = [s[1]['img_path'] for s in z.iterrows() 
                     if ((u'клара' in s[1]['ocrl'].lower() or u'кпара' in s[1]['ocrl'])
                         and (u'одаток' in s[1]['ocrl']))]
                 
#                          and (s[1]['img_path'].endswith('.0.png') or s[1]['img_path'].endswith('.1.png')) 
#                          and (not u'ларант' in s[1]['ocr'] and not u'парант' in s[1]['ocr'])
#                         )])
imgs = [x[:x[:-4].rfind('.')+1]+str(int(x[x[:-4].rfind('.')+1:-4]) + 1)+".png" for x in imgs]
pgt.import_list_into_pg('decl_imgs', imgs, col_name='img_path')
# imagesList=''.join( ["<img style='height: 200px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % 
#                      str(s.replace('/Users/tilarids/dev/decl/', '')) for s in imgs]) 
# display(HTML(imagesList))

In [16]:
from extract_util import four_point_transform

In [17]:
import cv2
from extract_util import extract_fields_given_borders

z = pgt.pg_query("""
    SELECT
        db.data->>'img_path' as img_path,
        db.data->'left_border' as left_border,
        db.data->'top_border' as top_border,
        db.data->'right_border' as right_border,
        db.data->'bottom_border' as bottom_border,
        db.data->'detected_width' as detected_width,
        db.data->'detected_height' as detected_height,
        db.data->'height' as height,
        db.data->'width' as width
    FROM detected_borders db
    INNER JOIN decl_imgs di on (di.img_path=db.data->>'img_path')
""")

for s in log_progress(z.iterrows(), every=1, size=len(z)):
    img_path = s[1]['img_path']
    try:
        tt = None
        tt = pgt.pg_query_by_id('extract_imgs', img_path)
    except:
        pass
    if tt:
        continue
    
    img = cv2.imread(img_path)
    record = extract_fields_given_borders(img, s[1]['left_border'], s[1]['top_border'], s[1]['right_border'], s[1]['bottom_border'])
    if record is None or not record['threshold_met']:
        continue
    for k,v in record.iteritems():
        if isinstance(v, np.ndarray):
            if k.endswith('_bin'):
                bin_path = s[1]['img_path'].replace('/img_data/', '/extract_img_data/') + "." + k.replace('_','.')

                np.save(bin_path, v)
                record[k] = bin_path + ".npy"
            else:
                png_path = s[1]['img_path'].replace('/img_data/', '/extract_img_data/') + "." + k + ".png"
                cv2.imwrite(png_path, v)
                record[k] = png_path
    record['img_path'] = img_path
    pgt.import_json_into_pg("extract_imgs", [record], lambda record: record['img_path'], create_table=False, skip_duplicates=True)            


In [18]:
extracts = pgt.pg_query("""
    SELECT
        ei.data->>'second_bin' as second_bin,
        ei.data->>'first_bin' as first_bin,
        ei.data->'first_width' as first_width,
        ei.data->'first_height' as first_height,
        ei.data->'second_width' as second_width,
        ei.data->'second_height' as second_height,
        ei.data->'img_path' as img_path,
        d.data->'income'->'5'->>'value' as income,
        d.data->'income'->'5'->>'family' as family_income,
        d.data->'path' as path
    FROM extract_imgs ei
    INNER JOIN detected_borders db on (db.data->>'img_path'=ei.data->>'img_path')
    INNER JOIN decls d on (db.data->>'path'=d.data->>'path')
""")
# len(extracts)
extracts[['income','family_income','first_bin','second_bin']].to_json('model_input.json',orient='records')
# extracts['second_width'].value_counts()
# extracts['first_height'].hist(bins=20)

In [ ]:
def show_extracted_borders(img_path):
    img = cv2.imread(img_path)
    left_border, top_border, right_border, bottom_border = extract_borders(img)
    cv2.line(img,(left_border[1],left_border[2]),(left_border[3],left_border[4]),(0,0,255),2)
    cv2.line(img,(top_border[1],top_border[2]),(top_border[3],top_border[4]),(0,255,0),2)
    cv2.line(img,(right_border[1],right_border[2]),(right_border[3],right_border[4]),(255,0,0),2)
    cv2.line(img,(bottom_border[1],bottom_border[2]),(bottom_border[3],bottom_border[4]),(255,0,255),2)
    print top_border
    print img.shape
    plt.figure()
    plt.imshow(img)

In [ ]:
jobs.status()

In [ ]:
pgt.import_json_into_pg("full_extract_imgs", [], lambda record: record['img_path'], create_table=True, skip_duplicates=True)

In [ ]:
# pgt.pg_query_by_id('decls','vulyk_7_82')['income']['5']

In [ ]:
841*0.01

In [ ]:
# !open /Users/tilarids/dev/decl/data/fc2dfe309d880193fd04672ff6c98ed664e78268e9dd26ba1608cc33.pdf

In [ ]:
jobs.traceback()

In [ ]:
jobs.new(extract_fields)

In [ ]:
import cv2
from extract_util import extract_fields_given_borders
import copy

all_second_pages = pgt.pg_query("""
    SELECT
        db.data->>'img_path' as img_path,
        db.data->'left_border' as left_border,
        db.data->'top_border' as top_border,
        db.data->'right_border' as right_border,
        db.data->'bottom_border' as bottom_border,
        db.data->'detected_width' as detected_width,
        db.data->'detected_height' as detected_height,
        db.data->'height' as height,
        db.data->'width' as width
    FROM detected_borders db
    where db.data->>'img_path' like '%.1.png'
""")
for s in log_progress(all_second_pages.iterrows(), every=1, size=len(all_second_pages)):
    img_path = s[1]['img_path']
    try:
        tt = None
        tt = pgt.pg_query_by_id('full_extract_imgs', img_path)
    except:
        pass
    if tt:
        continue
    
    img = cv2.imread(img_path)
    record = extract_fields_given_borders(img, s[1]['left_border'], s[1]['top_border'], s[1]['right_border'], s[1]['bottom_border'])
    if record is None: 
        continue
    out_record = {}
    out_record['img_path'] = img_path
    
    for k,v in record.iteritems():
        if isinstance(v, np.ndarray):
            out_record['source_key'] = k
            if k.endswith('_bin'):
                bin_path = img_path.replace('/img_data/', '/full_extract_img_data/') + "." + k.replace('_','.')

                np.save(bin_path, v)
                out_record['bin_path'] = bin_path + ".npy"
            else:
                ppm_path = img_path.replace('/img_data/', '/full_extract_img_data/') + "." + k + ".ppm"
                cv2.imwrite(ppm_path, v)
                out_record['ppm_path'] = ppm_path
    pgt.import_json_into_pg("full_extract_imgs", [out_record], lambda record: record['ppm_path'], create_table=False, skip_duplicates=True)            

In [ ]:
pgt.pg_query("""
    SELECT 
        fei.data->>'bin_path' as bin_path,
        fei.data->>'label' as label
    FROM full_extract_imgs fei
    WHERE 
        not fei.data->>'label' is null
  """).to_json('digits_classifier_input.json',orient='records')

In [ ]:
import cv2
for x in pgt.pg_query("""
    SELECT 
        fei.data->>'ppm_path' as ppm_path
    FROM full_extract_imgs fei
    WHERE 
        fei.data->>'label'='space' limit 10
  """)['ppm_path']:
    plt.figure()
    plt.imshow(cv2.imread(x))

In [28]:
jobs.status()

Running jobs:
0 : <function extract_fields at 0x106930ed8>
2 : <function apply_tesseract at 0x106930c08>

